# M1M3 Force Actuator following errors FFT

This analysis will go through all slews of a given day_obs, finding peaks above a certain threshold_peak in the FFT and that have a higher frequency than threshold_freq. 

We recommend that the first time you run the code you select the day you want to analyze and set slew_selector = False and plot_output=False. The code will give a first pass and return the anomalies found on each slew.

Once a slew of interest (slew_nb) is identified, the code can be rerun with slew_selector set to True (set slew_nb as well) and plot_output set to True. 

Note that the run time is 3 seconds per slew on average. More or less time depending on how many anomalies are found.

In [ ]:
# Times Square parameters
day_obs = 20250527
threshold_peak = 800
threshold_freq = 3.0 # Hz
slew_selector = False
slew_nb = 0
plot_output = False

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from astropy.time import Time
from scipy.signal import find_peaks

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import EfdClient, getEfdData, makeEfdClient, getDayObsEndTime, getDayObsStartTime
from lsst.ts.xml.tables.m1m3 import FATable

In [ ]:
def read_slews(day_obs):
    # Select data from a given date
    eventMaker = TMAEventMaker()
    events = eventMaker.getEvents(day_obs)

    # Get lists of slew and track events
    slews = [e for e in events if e.type == TMAState.SLEWING]
    print(f"Found {len(slews)} slews")
    return slews

In [ ]:
def loop_over_slews(slews):

    for s,slew in enumerate(slews):
        if slew_selector:
            if s != slew_nb:
                continue
        else:
            print(f'{s}/{len(slews)}')
        loop_over_actuators(client, s, slew)


In [ ]:
def loop_over_actuators(client, slew_nb, slew):

    FA_error = [f"primaryCylinderFollowingError{i}" for i in range(len(FATable))]
    FA_error += [f"secondaryCylinderFollowingError{i}" for i in range(112)]
    df = getEfdData(
        client,
        "lsst.sal.MTM1M3.forceActuatorData",
        columns=FA_error,
        begin=slew.begin,
        end=slew.end,
    )
    if len(df) < 10: #require minimum length
        print(f"Skipping slew {slew_nb} with few data points")
        return
    
    for fa in range(len(FATable)):
        actuator_id = FATable[fa].actuator_id
        df_ref = df[f'primaryCylinderFollowingError{fa}']
        fft_plot(slew_nb, df_ref, actuator_id, "primary", slew.begin, slew.end)
        if FATable[fa].s_index is None:
            continue
        else:
            df_ref = df[f'secondaryCylinderFollowingError{FATable[fa].s_index}']
            fft_plot(slew_nb, df_ref, actuator_id, "secondary", slew.begin, slew.end)
            

In [ ]:
def fft_plot(slew_nb, df_ref, actuator_id, actuator_type, t_start, t_end):

    plot_directory = "./plots/"
    if not os.path.exists(plot_directory):
        os.makedirs(plot_directory)
    dt = (df_ref.index[1] - df_ref.index[0]).total_seconds()
    freqs = np.fft.fftfreq(len(df_ref), d=dt)
    positive_mask = freqs > 0
    fft_frequency = freqs[positive_mask]
    fft_result = np.fft.fft(df_ref.values, axis=0)
    fft_magnitudes = np.abs(fft_result[positive_mask])
    peaks, properties = find_peaks(fft_magnitudes, height=10, distance=5)
    if len(peaks) < 1:
        print(f"Could not find peaks in FFT for FA following errors in {actuator_type} actuator {actuator_id}, in slew {slew_nb}")
        return
    # Extract peak frequencies and magnitudes
    peak_freqs = fft_frequency[peaks]
    peak_magnitudes = fft_magnitudes[peaks]
    largest = pd.Series(peak_magnitudes).max()
    max_freq = peak_freqs[pd.Series(peak_magnitudes).idxmax()]
    if ((largest > threshold_peak) and (max_freq > threshold_freq)): 
        print(f"Found anomaly in {actuator_type} actuator {actuator_id} in slew {slew_nb} above {threshold_peak}, frequency {max_freq:.2f} Hz {largest:.1f}")
        if plot_output:
            plt.figure(figsize=(10, 5))
            label = f"{actuator_type} actuator {actuator_id}"
            plt.plot(
                fft_frequency, fft_magnitudes, label=label
            )
            start = t_start.to_datetime().strftime('%H:%M:%S')
            end = t_end.to_datetime().strftime('%H:%M:%S')
            plt.title(f"{actuator_type} actuator {actuator_id}, slew {slew_nb} on {day_obs}. Power spectrum for {start} - {end}")
            plt.xlabel("Frequency [Hz]")
            plt.ylabel("Magnitude")
            #plt.legend()
            #plt.savefig(f"{plot_directory}PA_{actuator_id}_{slew_nb}_{day_obs}.png")
            plt.show()
            plt.close()

            plt.figure(figsize=(10,5))
            plt.plot(df_ref, label=label)
            plt.title(f"{actuator_type} actuator {actuator_id}, slew {slew_nb} on {day_obs}. Power spectrum for {start} - {end}")
            plt.xlabel("UTC")
            plt.ylabel("FA following error")
            #plt.legend()
            #plt.savefig(f"{plot_directory}PA_{actuator_id}_{slew_nb}_{day_obs}_FAerror.png")
            plt.show()
            plt.close()


In [ ]:
def main(day_obs):

    slews = read_slews(day_obs)

    loop_over_slews(slews)
        

In [ ]:
# Create an EFD client instance
client = makeEfdClient()

# Run for day_obs
main(day_obs)
